In [1]:
import numpy as np
import collections
import meshio

In [2]:
#meshio internal cell storage unit
CellBlock = collections.namedtuple("CellBlock", ["type", "data"])

#Compute average every n-th row
def groupedAvg(myArray, N=2):
    result = np.cumsum(myArray, 0)[N-1::N]/float(N)
    result[1:] = result[1:] - result[:-1]
    return result

In [9]:
eleType='tetra' #only support tetra mesh yet
mesh_file=r"test.msh"

#---------Read mesh and compute element centers----------
mesh = meshio.read(mesh_file, file_format="gmsh")
Dim=3 if abs(mesh.points[0][2])>1e-10 else 2
numEleVerts=mesh.cells_dict[eleType].shape[1]
print(mesh,'\n  Mesh dimension=',Dim,'\n  NumVerts of a %s element=' %(eleType),numEleVerts)
nodes=mesh.points
eles=mesh.cells_dict[eleType] 

#Compute element centroids [NumEle*4,3], we have element coords every 4 row
eles_center=groupedAvg(nodes[eles].reshape(-1,3),numEleVerts) 

<meshio mesh object>
  Number of points: 1358
  Number of cells:
    triangle: 1780
    tetra: 5319
  Cell data: gmsh:physical, gmsh:geometrical 
  Mesh dimension= 3 
  NumVerts of a tetra element= 4


- hilbertSort(arg0: int, arg1: numpy.ndarray[float64[m, n], flags.writeable, flags.c_contiguous], arg2: bool) -> Tuple[numpy.ndarray[float64[m, n]], array]
    
    
    Sort 1/2/3D points based on Hilbert curve
    
    Input Args:
    ----------- 
    1. [Int] Dimension 
    2. [Numpy.Array Nx3] InputPoints 
    3. [Bool] writeConnectivityMap, if output original/sorted points connectivity map into VTK
    
    Author:Bin Wang (binwang.0213@gmail.com)
    Date: April, 2020

In [10]:
#-----------Sort points and elements based on Hilbert Curve----------
import pyHilbertSort as sorter
nodes_sorted,nodeSortedIdx=sorter.hilbertSort(Dim,nodes)
center_sorted,eleSortedIdx=sorter.hilbertSort(Dim,eles_center)

nodes_sorted=nodes[nodeSortedIdx,:]
eles_sorted=eles[eleSortedIdx,:]

[Info]	Sorting 1358 points.......
	Bbox= -1.6  -1.6 -3.75  1.6  1.6 1.25
	SortTime=0 ms
	Done!
[Info]	Sorting 5319 points.......
	Bbox=-1.47145 -1.47077 -3.68276 1.47045 1.47167 1.19003
	SortTime=0 ms
	Done!


In [11]:
nodeSortedIdx=sorter.hilbertSortIdx(Dim,nodes)
eleSortedIdx=sorter.hilbertSortIdx(Dim,eles_center)

nodes_sorted=nodes[nodeSortedIdx,:]
eles_sorted=eles[eleSortedIdx,:]

[Info]	Sorting 1358 points.......
	Bbox= -1.6  -1.6 -3.75  1.6  1.6 1.25
	SortTime=0 ms
	Done!
[Info]	Sorting 5319 points.......
	Bbox=-1.47145 -1.47077 -3.68276 1.47045 1.47167 1.19003
	SortTime=0 ms
	Done!


In [12]:
sorter.writeConnectivityMap2VTK("OrigPts_node.vtk",nodes)
sorter.writeConnectivityMap2VTK("SorttedPts_node.vtk",nodes_sorted)
sorter.writeConnectivityMap2VTK("OrigPts_ele.vtk",eles_center)
sorter.writeConnectivityMap2VTK("SorttedPts_ele.vtk",eles_center[eleSortedIdx,:])

[Info]	Write point ordering map to OrigPts_node.vtk
[Info]	Write point ordering map to SorttedPts_node.vtk
[Info]	Write point ordering map to OrigPts_ele.vtk
[Info]	Write point ordering map to SorttedPts_ele.vtk


### Reordering cell only

In [13]:
#--------Update mesh info ----------
for i,cell in enumerate(mesh.cells):
    if cell.type == eleType:
        mesh.cells[i]=CellBlock(eleType, eles_sorted)

#--------Write updated mesh to file--------
mesh.write("test_sortted.msh",file_format="gmsh22",binary=False)

### Further reordering nodes and replace cell node index

In [14]:
NumPts=len(mesh.points)
mesh.points=nodes_sorted
idx_mapping = dict(zip(nodeSortedIdx,np.arange(NumPts)))

#Replace old idx by sorted idx
for i,cell in enumerate(mesh.cells):
    c_type,c_data=cell
    ##https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key
    c_data=np.vectorize(idx_mapping.get)(c_data) 
    mesh.cells[i]=CellBlock(c_type, c_data)
    
#--------Write updated mesh to file--------
mesh.write("test_sortted.msh",file_format="gmsh22",binary=False)

In [15]:
np.savetxt(r"..\visual_studio\Pts.csv", eles_center, delimiter=",")